In [ ]:

import time
import numpy as np
import csv
import pandas as pd
from google.colab import files
np.random.seed(0)

In [ ]:
d = 730
n = 1068
A_train = pd.read_csv('A1.csv', header=None).to_numpy()
A_val = pd.read_csv('A2.csv', header=None).to_numpy()

b_train = pd.read_csv('b1.csv', header=None).to_numpy().reshape(-1)
b_val = pd.read_csv('b2.csv', header=None).to_numpy().reshape(-1)

In [ ]:
import cvxpy as cp
lbd = 10
def gradg(x,S):
  S = list(S)
  AS = A_train[S,:]
  bS = b_train[S]
  return 2*AS.T@(AS@x-bS)/len(S)

def gradf(x,S):
  S = list(S)
  AS = A_val[S,:]
  bS = b_val[S]
  return 2*AS.T@(AS@x-bS)/len(S)

def g(x,S):
  S = list(S)
  AS = A_train[S,:]
  bS = b_train[S]
  return np.linalg.norm(AS@x-bS)**2/len(S)

def f(x,S):
  S = list(S)
  AS = A_val[S,:]
  bS = b_val[S]
  return np.linalg.norm(AS@x-bS)**2/len(S)

def lmo(c):
  i = np.argmax(abs(c))
  v = np.zeros(d)
  v[i] = -np.sign(c[i])*lbd
  return v

def clmo(c,g,h):
  v = cp.Variable(d)
  prob = cp.Problem(cp.Minimize(c.T@v),[cp.norm(v,1) <= lbd, g.T@v <= h])
  prob.solve()
  return v.value

In [ ]:
def simplex_projection(s):
  """Projection onto the unit simplex."""
  if np.sum(s) <=1 and np.all(s >= 0):
      return s

  u = np.sort(s)[::-1]
  cssv = np.cumsum(u)
  rho = np.nonzero(u * np.arange(1, len(u)+1) > (cssv - 1))[0][-1]
  theta = (cssv[rho] - 1) / (rho + 1.0)

  return np.maximum(s-theta, 0)

def proj(c):
  sol = simplex_projection(np.abs(c)/lbd)
  sol = lbd*np.sign(c)*sol
  return sol

In [ ]:
def spiderfw(init, budget):
  x = init
  t = 0
  q = np.ceil(np.sqrt(n/3))
  S = q
  oracle = 0
  while t <= budget:

   eta = 0.1/(t+2)
   if t%q == 0:
      ind = range(int(n/3))
      hat_gradg = gradg(x, ind)
      oracle += n
   else:
      ind = np.random.randint(0, n/3-1, size=int(S))
      hat_gradg = hat_gradg + gradg(x,ind) - gradg(x_prev,ind)
      oracle += S

   v = lmo(hat_gradg)
   x_prev = x
   x = x+eta*(v-x)
   #print('iteration: {}, inner-level: {}'.format(t, g(x,range(int(n/3)))))
   t += 1
  return x, oracle




In [ ]:

x = cp.Variable(d)
inner = cp.Problem(cp.Minimize(cp.sum_squares(A_train@x-b_train)),[cp.norm(x,1) <= lbd])
inner.solve()
g_opt = inner.value/(n/3)
print('Inner optimal value: {}'.format(g_opt))



In [ ]:
x = cp.Variable(d)
outer = cp.Problem(cp.Minimize(cp.sum_squares(A_val@x-b_val)),[cp.norm(x,1) <= lbd, cp.sum_squares(A_train@x-b_train)/(n/3) <=g_opt])
outer.solve()
f_opt = outer.value/(n/3)
print('Outer optimal value: {}'.format(f_opt))



In [ ]:
def irscg(init, budget,i):
  filename = 'irscg1_{}.csv'.format(i)
  # with open(filename, 'w', newline='') as file:
  #   writer = csv.writer(file)
  #   writer.writerow(["t","g_x","f_x","g_z","f_z","time","time_elapsed"])
  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []
  oracle = 0
  count = 1

  t = 0

  x = init
  varsigma = 10
  S = 1
  z = 0
  s = 0
  elapsed_time = 0
  while elapsed_time <= budget:

    start = time.time()

    alpha = 2/(t+2)
    sigma = varsigma*(t+1)**(-0.25)

    xi    = np.random.randint(0, n/3-1, size=int(S))
    theta = np.random.randint(0, n/3-1, size=int(S))


    if t == 0:

      hat_gradf = gradf(x, theta)
      hat_gradg = gradg(x, xi)

    else:

      hat_gradf = (1-alpha)*hat_gradf + gradf(x, theta) - (1-alpha)*gradf(x_prev, theta)
      hat_gradg = (1-alpha)*hat_gradg +  gradg(x, xi)   - (1-alpha)* gradg(x_prev, xi)

    v = lmo(sigma*hat_gradf +hat_gradg)
    x_prev = x
    x = x+alpha*(v-x)

    s_prev = s
    s = s+(t+1)*sigma
    z = (s_prev*z-(t+1)*t*sigma*x_prev+(t+2)*(t+1)*sigma*x)/s
    t += 1

    end = time.time()

    elapsed_time += end-start
    oracle += 2
    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(x,range(int(n/3)))-g_opt)
      outer.append(f(x,range(int(n/3)))-f_opt)
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1
  print(t)
  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)




def irfscg(init, budget,i):
  filename = 'irfscg1_{}.csv'.format(i)

  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []
  oracle = 0
  count = 1

  t = 0
  q = np.floor((n/3)**0.5)
  S = q
  x = init
  z = 0
  s = 0
  varsigma = 10

  elapsed_time = 0
  while elapsed_time <= budget:

    start = time.time()
    if t < q:
      alpha = np.log(q)/q
    else:
      alpha = 2/(t+2)

    sigma = varsigma*(max(t,q)+1)**(-0.5)

    if t%q == 0:
      ind = range(int(n/3))
      hat_gradf = gradf(x, ind)
      hat_gradg = gradg(x, ind)

    else:
      ind1 = np.random.randint(0, n/3-1, size=int(S))
      ind2 = np.random.randint(0, n/3-1, size=int(S))
      hat_gradf = hat_gradf + gradf(x,ind1) - gradf(x_prev,ind1)
      hat_gradg = hat_gradg + gradg(x,ind2) - gradg(x_prev,ind2)

    v = lmo(sigma*hat_gradf +hat_gradg)
    x_prev = x
    x = x+alpha*(v-x)

    s_prev = s
    s = s+(t+1)*sigma
    z = (s_prev*z-(t+1)*t*sigma*x_prev+(t+2)*(t+1)*sigma*x)/s
    t += 1

    end = time.time()

    elapsed_time += end-start
    if (t-1)%q == 0:
      oracle += 2*n
    else:
      oracle += 2*S

    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(x,range(int(n/3)))-g_opt)
      outer.append(f(x,range(int(n/3)))-f_opt)
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1
  print(t)
  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)

In [ ]:
def sbcgi(init,budget,i):
  filename = 'sbcgi1_{}.csv'.format(i)

  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []
  oracle = 0
  count = 1

  start = time.time()
  x, oracle = spiderfw(init, 10**5)
  g0 = g(x,range(int(n/3)))
  end = time.time()
  elapsed_time = end-start
  t = 0
  while elapsed_time <= budget:

    start = time.time()


    alpha = 0.01/(t+1)

    xi    = np.random.randint(0, n/3-1, size=1)
    theta = np.random.randint(0, n/3-1, size=1)


    if t == 0:

      hat_gradf = gradf(x, theta)
      hat_gradg = gradg(x, xi)
      hat_g     = g(x,xi)

    else:

      hat_gradf = (1-alpha)*hat_gradf + gradf(x, theta) - (1-alpha)*gradf(x_prev, theta)
      hat_gradg = (1-alpha)*hat_gradg +  gradg(x, xi)   - (1-alpha)* gradg(x_prev, xi)
      hat_g     = (1-alpha)* hat_g    +    g(x, xi)     - (1-alpha)*   g(x_prev, xi)

    v = clmo(hat_gradf, hat_gradg, hat_gradg.T@x+g0-hat_g+10**(-4)/np.sqrt(t+1))
    x_prev = x
    x = x+alpha*(v-x)
    t += 1

    end = time.time()

    elapsed_time += end-start
    oracle += 2
    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(x,range(int(n/3)))-g_opt)
      outer.append(f(x,range(int(n/3)))-f_opt)
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1

  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)

def sbcgf(init, budget, i):
  filename = 'sbcgf1_{}.csv'.format(i)

  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []
  oracle = 0
  count = 1

  S = np.floor((n/3)**0.5)
  q = np.floor((n/3)**0.5)

  start = time.time()
  x, oracle = spiderfw(init, 10**5)
  g0 = g(x,range(int(n/3)))
  end = time.time()
  elapsed_time = end-start
  t = 0
  while elapsed_time <= budget:

    start = time.time()

    alpha = 10**(-5)

    if t%q == 0:
      ind = range(int(n/3))
      hat_gradf = gradf(x, ind)
      hat_gradg = gradg(x, ind)
      hat_g     =   g(x, ind)

    else:
      ind1 = np.random.randint(0, n/3-1, size=int(S))
      ind2 = np.random.randint(0, n/3-1, size=int(S))
      hat_gradf = hat_gradf + gradf(x,ind1) - gradf(x_prev,ind1)
      hat_gradg = hat_gradg + gradg(x,ind2) - gradg(x_prev,ind2)
      hat_g     =  hat_g    +   g(x, ind2)   -   g(x_prev, ind2)

    v = clmo(hat_gradf, hat_gradg, hat_gradg.T@x+g0-hat_g+10**(-4)/np.sqrt(t+1))
    x_prev = x
    x = x+alpha*(v-x)
    t += 1

    end = time.time()

    elapsed_time += end-start
    if (t-1)%q == 0:
      oracle += 2*n
    else:
      oracle += 2*S

    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(x,range(int(n/3)))-g_opt)
      outer.append(f(x,range(int(n/3)))-f_opt)
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1

  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)

In [ ]:
def aripseg(init, budget, short,i):
  if short == True:
    gamma0 = 10**(-7)
    filename = 'aripseg_short1_{}.csv'.format(i)
  else:
    gamma0 = 10**(-2)
    filename = 'aripseg_long1_{}.csv'.format(i)


  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []
  oracle = 0
  count = 1

  t = 0
  x = init

  rho0 = 10**3
  Gamma = 0
  r = 0.5
  elapsed_time = 0
  while elapsed_time <= budget:

    start = time.time()

    gamma = gamma0/(t+1)**0.75
    rho = rho0*(t+1)**0.25

    xi    = np.random.randint(0, n/3-1, size=1)
    theta = np.random.randint(0, n/3-1, size=1)

    hat_gradf = gradf(x,xi)
    hat_gradg = gradg(x,theta)

    y = proj(x-gamma*(hat_gradf+rho*hat_gradg))

    xi    = np.random.randint(0, n/3-1, size=1)
    theta = np.random.randint(0, n/3-1, size=1)

    hat_gradf = gradf(y,xi)
    hat_gradg = gradg(y,theta)

    x = proj(x-gamma*(hat_gradf+rho*hat_gradg))

    Gamma_prev = Gamma
    Gamma = Gamma+(gamma*rho)**r

    if t == 0:
      bar_y = y
    else:
      bar_y = (Gamma_prev*bar_y+((gamma*rho)**r)*y)/Gamma
    t += 1
    end = time.time()

    elapsed_time += end-start
    oracle += 4
    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(bar_y,range(int(n/3)))-g_opt)
      outer.append(f(bar_y,range(int(n/3)))-f_opt)
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1

  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)

def dbgdsto(init, budget,short ,i):

  if short == True:
    gamma0 = 10**(-6)
    filename = 'dbgdsto_short1_{}.csv'.format(i)
  else:
    gamma0 = 10**(-2)
    filename = 'dbgdsto_long1_{}.csv'.format(i)


  iters = []
  inner = []
  outer = []
  time_elapsed = []
  oracles = []
  oracle = 0
  count = 1

  t = 0
  x = init
  alpha = 1
  beta = 1
  gamma = 10**(-6)
  elapsed_time = 0
  while elapsed_time <= budget:

    start = time.time()

    xi    = np.random.randint(0, n/3-1, size=1)
    theta = np.random.randint(0, n/3-1, size=1)

    hat_gradf = gradf(x,xi)
    hat_gradg = gradg(x,theta)
    hat_g = g(x,theta)

    norm_sq = np.linalg.norm(hat_gradg)**2
    hat_phi = min(alpha*(hat_g), beta*np.linalg.norm(norm_sq)**2)
    nu = max((hat_phi-hat_gradf.T@hat_gradg)/norm_sq, 0)
    x = proj(x -gamma*(hat_gradf+nu*hat_gradg))

    t += 1
    end = time.time()

    elapsed_time += end-start
    oracle += 2
    if elapsed_time >= count:
      iters.append(t)
      inner.append(g(x,range(int(n/3)))-g_opt)
      outer.append(f(x,range(int(n/3)))-f_opt)
      time_elapsed.append(elapsed_time)
      oracles.append(oracle)
      count += 1

  df = pd.DataFrame({
    't': iters,
    'inner': inner,
    'outer': outer,
    'oracles': oracles,
    'time_elapsed': time_elapsed})
  df.to_csv(filename, index=False)
  files.download(filename)

In [ ]:
budget = 240
for i in range(10):
  init = np.random.rand(d)
  if np.linalg.norm(init,1) > lbd:
    init = init/np.linalg.norm(init,1)

  irscg(init, budget, i)
  irfscg(init, budget, i)
  sbcgi(init, budget, i)
  sbcgf(init, budget, i)
  aripseg(init, budget, True, i)
  aripseg(init, budget, False, i)
  dbgdsto(init, budget, True, i)
  dbgdsto(init, budget, False, i)